In [7]:
# !pip install libpysal

In [10]:
import pandas as pd
import numpy as np
import geopandas as gpd
import libpysal
import matplotlib.pyplot as plt
pd.set_option("display.max_columns", None)
import warnings
warnings.filterwarnings('ignore')

In [11]:
df = pd.read_parquet('Data/Cleaned/after_kriging.parquet')
labels = pd.read_parquet('Data/Cleaned/labels_udp.parquet')
df

,NAME,total_pop,percent_hh_income_assist,percent_hh_snap,percent_hh_english,percent_work_agriculture,percent_work_construction,percent_work_retail,percent_work_finance,percent_work_stem,percent_work_edu_health,percent_work_art,percent_out_of_county,percent_moved_within_county,percent_public_transit,percent_drive_commute,percent_bike_commute,percent_walk_commute,percent_wfh,percent_white,percent_black,percent_native,percent_asian,percent_pacific,percent_latino,percent_bachelors,percent_grad,percent_born_citizen,percent_naturalized_citizen,percent_not_citizen,percent_same_house_1yr,percent_out_of_state,med_income,med_rent_acs,med_age,percent_hh_rented,med_value,percent_hh_electric,percent_hh_solar,year,med_hh_age,med_hh_tenure,full_tract,GEOID,num_citibike_rides,num_trees,med_rent,num_affordable_hous_built,num_bikelanes,num_evictions_commercial,num_evictions_residential,park_acres,num_parks,random_lat,random_lon
0,"Census Tract 1, Bronx County, New York",11517.000000,0.051048,0.223460,0.310196,0.001382,0.050642,0.122766,0.095326,0.095825,0.243203,0.100514,0.583138,0.191282,0.000000,0.000000,0.000000,0.000000,0.000000,0.140662,0.564557,0.001216,0.018755,0.000000,0.348962,0.014066,0.001823,0.763220,0.044022,0.110793,0.194235,0.030564,43837.207255,1177.485307,31.400000,0.001878,462621.097233,0.098306,0.000000,2010,78.695256,24.42752,005000100,36005000100,0.0,441.302083,2063.183528,0.0,0.0,0.308626,5.238882,0.000,0.0,227454.007430,1.017060e+06
1,"Census Tract 2, Bronx County, New York",4286.000000,0.039322,0.102544,0.265227,0.000000,0.083885,0.192053,0.113687,0.080574,0.229581,0.045254,0.016799,0.049930,0.135091,0.269482,0.000000,0.006300,0.007933,0.145124,0.309146,0.008866,0.042697,0.000000,0.715119,0.059729,0.023098,0.648157,0.170089,0.067429,0.905973,0.003733,59826.000000,1273.000000,29.000000,0.004626,392500.000000,0.021588,0.000000,2010,77.000000,23.00000,005000200,36005000200,0.0,308.000000,2402.978322,0.0,2.0,0.000000,0.000000,272.835,4.0,234078.862014,1.023593e+06
2,"Census Tract 4, Bronx County, New York",4893.000000,0.003919,0.099935,0.308295,0.000000,0.075145,0.047481,0.114368,0.087531,0.384393,0.029315,0.036787,0.055590,0.192520,0.274474,0.003679,0.012058,0.000000,0.114654,0.230738,0.000000,0.023503,0.000000,0.715308,0.103617,0.061312,0.680564,0.103209,0.087472,0.882690,0.000000,71968.000000,1330.000000,36.100000,0.064010,393700.000000,0.008491,0.000000,2010,47.000000,23.00000,005000400,36005000400,0.0,318.000000,2325.892252,0.0,17.0,0.000000,4.250000,362.538,6.0,232916.392053,1.025947e+06
3,"Census Tract 16, Bronx County, New York",5140.000000,0.037821,0.222895,0.379223,0.000000,0.050652,0.084253,0.038114,0.024574,0.451354,0.048646,0.003502,0.059533,0.172568,0.176459,0.000000,0.007004,0.006226,0.191245,0.357977,0.000000,0.000000,0.000000,0.621401,0.046109,0.042218,0.625097,0.138132,0.094553,0.924125,0.005837,30355.000000,823.000000,40.200000,0.022693,378100.000000,0.095310,0.000000,2010,57.000000,27.00000,005001600,36005001600,0.0,367.000000,2349.838702,0.0,12.0,0.000000,8.125000,68.147,2.0,238003.638656,1.023288e+06
4,None,4456.582103,0.090314,0.345251,0.425018,0.002133,0.057241,0.120698,0.061229,0.082515,0.292372,0.128216,0.031838,0.060921,0.250265,0.050562,0.003455,0.030181,0.006441,0.241556,0.435755,0.004220,0.026521,0.000213,0.532698,0.045966,0.015799,0.647966,0.073836,0.167280,0.882765,0.011179,30851.843530,671.242573,31.914041,0.010499,482173.640895,0.107019,0.000000,2010,62.219022,25.00895,005001901,36005001901,0.0,68.000000,2487.701369,2.4,15.0,0.000000,15.875000,0.000,0.0,233255.890204,1.004954e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2320,Census Tract 303.02; Richmond County; New York,6502.000000,0.073832,0.151869,0.403271,0.000000,0.044951,0.047557,0.041694,0.069381,0.424104,0.121824,0.010766,0.018917,0.131959,0.289449,0.000

## Per-Capita Features

In [12]:
# to_change = ['num_citibike_rides','num_trees', 'num_affordable_hous_built', 'num_bikelanes',
#        'num_evictions_commercial', 'num_evictions_residential','num_parks']
# for col in to_change:
#     name = col.replace('num_','')
#     df[name+'_pc'] = df[col]/df['total_pop']
# df.drop(to_change, axis=1, inplace=True)

## Generate Features for X-Year Percent Change
**We train on a 6 year period, and predict 4 years into the future**
- train on features 2010-2016 to predict gentrification status in 2020 (labels from 2020, eligible tracts from 2010)
- 2011-2017
- 2012-2018
- train on features 2013-2019 to predict gentrification status in 2023 (labels from 2023, eligible tracts from 2013)
- then use features 2017-2023 to predict gentrification status in 2027 (no labels, eligible tracts are taken from 2017)

In [13]:
df_10_16 = df[df.year.isin(['2010','2011','2012','2013','2014','2015','2016'])]  # training set
df_11_17 = df[df.year.isin(['2011','2012','2013','2014','2015','2016','2017'])]  # training set
df_12_18 = df[df.year.isin(['2012','2013','2014','2015','2016','2017','2018'])]  # training set
df_13_19 = df[df.year.isin(['2013','2014','2015','2016','2017','2018','2019'])]  # training set
df_14_20 = df[df.year.isin(['2014','2015','2016','2017','2018','2019','2020'])]  # pred set
df_15_21 = df[df.year.isin(['2015','2016','2017','2018','2019','2020','2021'])]  # pred set
df_16_22 = df[df.year.isin(['2016','2017','2018','2019','2020','2021','2022'])]  # pred set
df_17_23 = df[df.year.isin(['2017','2018','2019','2020','2021','2022','2023'])]  # pred set

df_10_16 = df_10_16.drop(['NAME','year', 'full_tract','random_lat', 'random_lon'], axis=1).groupby('GEOID').agg(lambda x: x.pct_change().replace([np.inf, -np.inf], np.nan).mean())
df_11_17 = df_11_17.drop(['NAME','year', 'full_tract','random_lat', 'random_lon'], axis=1).groupby('GEOID').agg(lambda x: x.pct_change().replace([np.inf, -np.inf], np.nan).mean())
df_12_18 = df_12_18.drop(['NAME','year', 'full_tract','random_lat', 'random_lon'], axis=1).groupby('GEOID').agg(lambda x: x.pct_change().replace([np.inf, -np.inf], np.nan).mean())
df_13_19 = df_13_19.drop(['NAME','year', 'full_tract','random_lat', 'random_lon'], axis=1).groupby('GEOID').agg(lambda x: x.pct_change().replace([np.inf, -np.inf], np.nan).mean())
df_14_20 = df_14_20.drop(['NAME','year', 'full_tract','random_lat', 'random_lon'], axis=1).groupby('GEOID').agg(lambda x: x.pct_change().replace([np.inf, -np.inf], np.nan).mean())
df_15_21 = df_15_21.drop(['NAME','year', 'full_tract','random_lat', 'random_lon'], axis=1).groupby('GEOID').agg(lambda x: x.pct_change().replace([np.inf, -np.inf], np.nan).mean())
df_16_22 = df_16_22.drop(['NAME','year', 'full_tract','random_lat', 'random_lon'], axis=1).groupby('GEOID').agg(lambda x: x.pct_change().replace([np.inf, -np.inf], np.nan).mean())
df_17_23 = df_17_23.drop(['NAME','year', 'full_tract','random_lat', 'random_lon'], axis=1).groupby('GEOID').agg(lambda x: x.pct_change().replace([np.inf, -np.inf], np.nan).mean())

df_10_16['pred_year']='2020'
df_11_17['pred_year']='2021'
df_12_18['pred_year']='2022'
df_13_19['pred_year']='2023'
df_14_20['pred_year']='2024'
df_15_21['pred_year']='2025'
df_16_22['pred_year']='2026'
df_17_23['pred_year']='2027'


In [14]:
df_train = pd.concat([df_10_16,df_11_17,df_12_18, df_13_19,df_14_20,df_15_21,df_16_22, df_17_23])

# drop rows that became entirely null bc of the % change
df = df_train.dropna(how='all', subset=[ 'percent_white', 'percent_black', 'percent_native',
       'percent_asian', 'percent_pacific', 'percent_latino', 'percent_bachelors',
       'percent_grad'])

df.fillna(0, inplace=True)
df.reset_index(inplace=True)
df

,GEOID,total_pop,percent_hh_income_assist,percent_hh_snap,percent_hh_english,percent_work_agriculture,percent_work_construction,percent_work_retail,percent_work_finance,percent_work_stem,percent_work_edu_health,percent_work_art,percent_out_of_county,percent_moved_within_county,percent_public_transit,percent_drive_commute,percent_bike_commute,percent_walk_commute,percent_wfh,percent_white,percent_black,percent_native,percent_asian,percent_pacific,percent_latino,percent_bachelors,percent_grad,percent_born_citizen,percent_naturalized_citizen,percent_not_citizen,percent_same_house_1yr,percent_out_of_state,med_income,med_rent_acs,med_age,percent_hh_rented,med_value,percent_hh_electric,percent_hh_solar,med_hh_age,med_hh_tenure,num_citibike_rides,num_trees,med_rent,num_affordable_hous_built,num_bikelanes,num_evictions_commercial,num_evictions_residential,park_acres,num_parks,pred_year
0,36005000100,-0.068661,0.035685,0.028985,0.001017,0.199477,0.045892,-0.017763,0.035731,0.022690,0.021284,0.054678,-0.014333,0.065448,0.000000,0.000000,0.000000,0.000000,0.000000,-0.006464,0.018035,0.277300,-0.000033,0.000000,-0.024075,-0.124830,0.449510,0.002733,0.025448,0.010323,-0.014150,-0.050057,0.005722,0.049193,0.004397,1.089373,0.008277,-0.005696,12.906525,-0.007878,-0.034566,0.0,0.040565,0.013164,0.000000,0.000000,0.144316,0.082033,0.0,0.0,2020
1,36005000200,0.034962,0.218237,0.201764,-0.026817,-0.211255,-0.143559,-0.051058,0.005439,0.075584,0.050176,0.071833,-0.077578,-0.105136,0.033405,-0.021183,0.000000,0.138279,-0.377954,0.214509,-0.026615,-0.298942,-0.017225,0.000000,0.009071,0.057625,0.148837,-0.027422,0.023382,0.149468,0.009212,-0.077517,0.030404,-0.003322,0.049306,1.106573,0.001691,0.311270,0.000000,-0.021834,-0.021378,0.0,0.039684,0.000384,0.000000,0.000000,0.000000,0.000000,0.0,0.0,2020
2,36005000400,0.034124,0.786830,0.168444,-0.006987,0.000000,-0.086546,0.085892,-0.119376,0.065526,-0.033501,0.236460,-0.107590,-0.004790,0.028070,-0.028604,-0.187822,-0.041648,2.037930,0.255629,0.070771,0.083326,-0.038641,0.000000,-0.016291,0.000209,0.032225,0.000973,0.054771,0.012176,0.007979,-0.229897,0.011434,0.009918,0.015270,-0.394797,-0.017437,1.049936,0.000000,-0.060346,-0.014822,0.0,0.065020,-0.002948,0.000000,0.070962,0.000000,0.051681,0.0,0.0,2020
3,36005001600,0.028238,0.105534,0.119549,-0.037455,0.000000,-0.068067,0.070625,-0.016419,0.270917,-0.013170,0.057567,0.232766,-0.104326,-0.015358,-0.022066,0.000000,0.312918,-0.095215,0.168040,0.005470,0.000000,0.000000,0.000000,0.013476,0.015757,-0.076782,-0.031302,0.045230,0.142282,0.004117,-0.403649,0.008068,0.023547,-0.013855,7.142181,0.003578,0.013124,0.000000,-0.017877,-0.040687,0.0,0.063187,0.029170,0.000000,0.069444,0.000000,0.106212,0.0,0.0,2020
4,36005001901,0.000644,-0.034693,0.006081,-0.001591,-0.081312,-0.047796,-0.014341,-0.055569,0.035664,-0.007433,0.090601,-0.102033,-0.018727,0.016370,-0.032029,0.232876,0.007768,0.041536,-0.064106,-0.018095,0.106253,-0.061774,0.422531,0.003428,0.049573,0.224663,0.004055,0.049740,-0.004173,0.002864,-0.171264,-0.018268,0.078083,0.001804,7.661378,-0.023169,-0.001365,-0.010289,0.026425,-0.039251,0.0,0.055470,0.016927,-0.520833,0.000000,0.000000,0.048778,0.0,0.0,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18595,36085030302,-0.001154,0.215473,0.011413,-0.021104,0.000000,-0.036738,-0.134240,-0.067538,0.016564,0.077345,0.148286,0.698988,-0.034859,0.041904,-0.004503,0.000000,0.209163,0.946175,-0.045150,0.086675,0.000000,0.011771,0.000000,0.008026,0.055342,0.008809,-0.009526,0.082499,-0.051934,0.002615,-0.173023,0.033478,0.025459,0.025387,-0.050505,0.064139,-0.006120,-0.066851,-0.011909,-0.032434,0.0,0.002710,0.018248,0.000000,0.000000,-1.000000,0.547619,0.0,0.0,2027
18596,36085031901,0.061577,-0.040771,-0.050215,-0.006222,0.000000,1.556189,0.042698,0.185330,0.093050,0.046607,0.215565,-0.330398

## Spatial Lag Feature
Feature to measure proximity to already-gentrified / super-gentrified tracts

In [15]:
# Load the census tract shapefile
gdf = gpd.read_file('Data/Raw/tracts2020_shapefile/nyct2020.shp')
gdf = gdf.to_crs(epsg = 4326)
gdf = gdf.merge(labels, on='GEOID')


In [16]:
# Create a weights matrix based on contiguity (e.g., Queen's case)
w = libpysal.weights.contiguity.Queen.from_dataframe(gdf)
    
# Row-standardize the weights (optional, but often recommended)
w.transform = 'R'

# Calculate the spatial lag
spatial_lag = libpysal.weights.spatial_lag.lag_spatial(w, gdf['vulnerable'])

# Add the spatial lag as a new column in the GeoDataFrame
gdf['lag_' + 'vulnerable'] = spatial_lag


# Identify observations with no neighbors
islands = w.islands
if islands:
    # Assign the original value to the spatial lag for islands
    gdf.loc[gdf.index.isin(islands), 'lag_' + 'vulnerable'] = gdf.loc[gdf.index.isin(islands), 'vulnerable']

lag_df = gdf[['NTAName','GEOID','vulnerable','pred_year','lag_vulnerable']]

In [17]:
# add spatial lag feature to the main df
df = df.merge(lag_df[['lag_vulnerable','GEOID','pred_year']], on=['GEOID','pred_year'])
df

,GEOID,total_pop,percent_hh_income_assist,percent_hh_snap,percent_hh_english,percent_work_agriculture,percent_work_construction,percent_work_retail,percent_work_finance,percent_work_stem,percent_work_edu_health,percent_work_art,percent_out_of_county,percent_moved_within_county,percent_public_transit,percent_drive_commute,percent_bike_commute,percent_walk_commute,percent_wfh,percent_white,percent_black,percent_native,percent_asian,percent_pacific,percent_latino,percent_bachelors,percent_grad,percent_born_citizen,percent_naturalized_citizen,percent_not_citizen,percent_same_house_1yr,percent_out_of_state,med_income,med_rent_acs,med_age,percent_hh_rented,med_value,percent_hh_electric,percent_hh_solar,med_hh_age,med_hh_tenure,num_citibike_rides,num_trees,med_rent,num_affordable_hous_built,num_bikelanes,num_evictions_commercial,num_evictions_residential,park_acres,num_parks,pred_year,lag_vulnerable
0,36005000100,-0.068661,0.035685,0.028985,0.001017,0.199477,0.045892,-0.017763,0.035731,0.022690,0.021284,0.054678,-0.014333,0.065448,0.000000,0.000000,0.000000,0.000000,0.000000,-0.006464,0.018035,0.277300,-0.000033,0.000000,-0.024075,-0.124830,0.449510,0.002733,0.025448,0.010323,-0.014150,-0.050057,0.005722,0.049193,0.004397,1.089373,0.008277,-0.005696,12.906525,-0.007878,-0.034566,0.0,0.040565,0.013164,0.000000,0.000000,0.144316,0.082033,0.0,0.0,2020,1.000000
1,36005000200,0.034962,0.218237,0.201764,-0.026817,-0.211255,-0.143559,-0.051058,0.005439,0.075584,0.050176,0.071833,-0.077578,-0.105136,0.033405,-0.021183,0.000000,0.138279,-0.377954,0.214509,-0.026615,-0.298942,-0.017225,0.000000,0.009071,0.057625,0.148837,-0.027422,0.023382,0.149468,0.009212,-0.077517,0.030404,-0.003322,0.049306,1.106573,0.001691,0.311270,0.000000,-0.021834,-0.021378,0.0,0.039684,0.000384,0.000000,0.000000,0.000000,0.000000,0.0,0.0,2020,0.574468
2,36005000400,0.034124,0.786830,0.168444,-0.006987,0.000000,-0.086546,0.085892,-0.119376,0.065526,-0.033501,0.236460,-0.107590,-0.004790,0.028070,-0.028604,-0.187822,-0.041648,2.037930,0.255629,0.070771,0.083326,-0.038641,0.000000,-0.016291,0.000209,0.032225,0.000973,0.054771,0.012176,0.007979,-0.229897,0.011434,0.009918,0.015270,-0.394797,-0.017437,1.049936,0.000000,-0.060346,-0.014822,0.0,0.065020,-0.002948,0.000000,0.070962,0.000000,0.051681,0.0,0.0,2020,0.723404
3,36005001600,0.028238,0.105534,0.119549,-0.037455,0.000000,-0.068067,0.070625,-0.016419,0.270917,-0.013170,0.057567,0.232766,-0.104326,-0.015358,-0.022066,0.000000,0.312918,-0.095215,0.168040,0.005470,0.000000,0.000000,0.000000,0.013476,0.015757,-0.076782,-0.031302,0.045230,0.142282,0.004117,-0.403649,0.008068,0.023547,-0.013855,7.142181,0.003578,0.013124,0.000000,-0.017877,-0.040687,0.0,0.063187,0.029170,0.000000,0.069444,0.000000,0.106212,0.0,0.0,2020,0.714286
4,36005001901,0.000644,-0.034693,0.006081,-0.001591,-0.081312,-0.047796,-0.014341,-0.055569,0.035664,-0.007433,0.090601,-0.102033,-0.018727,0.016370,-0.032029,0.232876,0.007768,0.041536,-0.064106,-0.018095,0.106253,-0.061774,0.422531,0.003428,0.049573,0.224663,0.004055,0.049740,-0.004173,0.002864,-0.171264,-0.018268,0.078083,0.001804,7.661378,-0.023169,-0.001365,-0.010289,0.026425,-0.039251,0.0,0.055470,0.016927,-0.520833,0.000000,0.000000,0.048778,0.0,0.0,2020,0.978723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18595,36085030302,-0.001154,0.215473,0.011413,-0.021104,0.000000,-0.036738,-0.134240,-0.067538,0.016564,0.077345,0.148286,0.698988,-0.034859,0.041904,-0.004503,0.000000,0.209163,0.946175,-0.045150,0.086675,0.000000,0.011771,0.000000,0.008026,0.055342,0.008809,-0.009526,0.082499,-0.051934,0.002615,-0.173023,0.033478,0.025459,0.025387,-0.050505,0.064139,-0.006120,-0.066851,-0.011909,-0.032434,0.0,0.002710,0.018248,0.000000,0.000000,-1.000000,0.547619,0.0,0.0,2027,0.297872
18596,36085031901,0.061577,-0.040771,-0.050215,-0.006222

In [18]:
# save features
# df.to_parquet('Data/Cleaned/post_feat_engineering.parquet')